# Ridge and Lasso Regression - Lab

## Introduction

In this lab, you'll practice your knowledge on Ridge and Lasso regression!

## Objectives

You will be able to:

- Use Lasso and ridge regression in Python
- Compare Lasso and Ridge with standard regression

## Housing Prices Data

Let's look at yet another house pricing data set.

In [1]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

df = pd.read_csv('Housing_Prices/train.csv')

Look at df.info

In [6]:
# Your code here
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
Id               1460 non-null int64
MSSubClass       1460 non-null int64
MSZoning         1460 non-null object
LotFrontage      1201 non-null float64
LotArea          1460 non-null int64
Street           1460 non-null object
Alley            91 non-null object
LotShape         1460 non-null object
LandContour      1460 non-null object
Utilities        1460 non-null object
LotConfig        1460 non-null object
LandSlope        1460 non-null object
Neighborhood     1460 non-null object
Condition1       1460 non-null object
Condition2       1460 non-null object
BldgType         1460 non-null object
HouseStyle       1460 non-null object
OverallQual      1460 non-null int64
OverallCond      1460 non-null int64
YearBuilt        1460 non-null int64
YearRemodAdd     1460 non-null int64
RoofStyle        1460 non-null object
RoofMatl         1460 non-null object
Exterior1st      1460 non-n

First, make a selection of the data by removing some of the data with `dtype = object`, this way our first model only contains **continuous features**

Make sure to remove the SalesPrice column from the predictors (which you store in `X`), then replace missing inputs by the median per feature.

Store the target in `y`.

In [7]:
# Load necessary packages
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import Lasso, Ridge, LinearRegression
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
# remove "object"-type features and SalesPrice from `X`
features = [col for col in df.columns if df[col].dtype in [np.float64, np.int64] and col!='SalePrice']
X =df[features]
# Impute null values
for col in X:
    med = X[col].median()
    X[col].fillna(value =med, inplace =True)

# Create y
y = df.SalePrice


Look at the information of `X` again

In [8]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 37 columns):
Id               1460 non-null int64
MSSubClass       1460 non-null int64
LotFrontage      1460 non-null float64
LotArea          1460 non-null int64
OverallQual      1460 non-null int64
OverallCond      1460 non-null int64
YearBuilt        1460 non-null int64
YearRemodAdd     1460 non-null int64
MasVnrArea       1460 non-null float64
BsmtFinSF1       1460 non-null int64
BsmtFinSF2       1460 non-null int64
BsmtUnfSF        1460 non-null int64
TotalBsmtSF      1460 non-null int64
1stFlrSF         1460 non-null int64
2ndFlrSF         1460 non-null int64
LowQualFinSF     1460 non-null int64
GrLivArea        1460 non-null int64
BsmtFullBath     1460 non-null int64
BsmtHalfBath     1460 non-null int64
FullBath         1460 non-null int64
HalfBath         1460 non-null int64
BedroomAbvGr     1460 non-null int64
KitchenAbvGr     1460 non-null int64
TotRmsAbvGrd     1460 non-null int64
F

## Let's use this data to perform a first naive linear regression model

Compute the R squared and the MSE for both train and test set.

In [10]:
from sklearn.metrics import mean_squared_error, mean_squared_log_error

# Split in train and test
X_train,X_test,y_train,y_test = train_test_split(X,y)
# Fit the model and print R2 and MSE for train and test
reg = LinearRegression()
reg.fit(X_train,y_train)

print('Training r^2:', reg.score(X_train, y_train))
print('Testing r^2:', reg.score(X_test, y_test))
print('Training MSE:', mean_squared_error(y_train, reg.predict(X_train)))
print('Testing MSE:', mean_squared_error(y_test, reg.predict(X_test)))

Training r^2: 0.8102405133777575
Testing r^2: 0.8039008670938336
Training MSE: 1189210459.7038302
Testing MSE: 1258935916.3074877


## Normalize your data

We haven't normalized our data, let's create a new model that uses `preprocessing.scale` to scale our predictors!

In [11]:
from sklearn import preprocessing

# Scale the data and perform train test split
X_scaled = preprocessing.scale(X)
X_train, X_test, y_train, y_test = train_test_split(X_scaled,y)


Perform the same linear regression on this data and print out R-squared and MSE.

In [12]:
# Your code here
reg.fit(X_train, y_train)

print('Training r^2:', reg.score(X_train, y_train))
print('Testing r^2:', reg.score(X_test, y_test))
print('Training MSE:', mean_squared_error(y_train, reg.predict(X_train)))
print('Testing MSE:', mean_squared_error(y_test, reg.predict(X_test)))

Training r^2: 0.8191464406326909
Testing r^2: 0.7826054064713207
Training MSE: 1171604235.8875566
Testing MSE: 1259202974.363278


## Include dummy variables

Your model hasn't included dummy variables so far: let's use the "object" variables again and create dummies

In [15]:
# Create X_cat which contains only the categorical variables
cat =[col for col in df.columns if df[col].dtype in [np.object]]
X_cat = df[cat]

X_cat.shape

(1460, 43)

In [16]:
# Make dummies
X_cat = pd.get_dummies(X_cat, drop_first = True)
X_cat.shape

(1460, 209)

Merge `x_cat` together with our scaled `X` so you have one big predictor dataframe.

In [17]:
type(X_scaled)

numpy.ndarray

In [18]:
# Your code here
X_all = pd.concat([pd.DataFrame(X_scaled), X_cat], axis =1)

In [29]:
len(X_all.columns)

246

Perform the same linear regression on this data and print out R-squared and MSE.

In [19]:
# Your code here
X_train, X_test, y_train, y_test = train_test_split(X_all,y)
linreg_all = LinearRegression()
linreg_all.fit(X_train, y_train)
print('Training R^2:', linreg_all.score(X_train, y_train))
print('Testing R^2:', linreg_all.score(X_test,y_test))
print('Training MSE:', mean_squared_error(y_train, linreg_all.predict(X_train)))
print('Testing MSE:', mean_squared_error(y_test, linreg_all.predict(X_test)))

Training R^2: 0.9377353758897593
Testing R^2: -4.841095672936132e+17
Training MSE: 405352795.0618151
Testing MSE: 2.743920558259091e+27


Notice the severe overfitting above; our training R squared is quite high, but the testing R squared is negative! Our predictions are far off. Similarly, the scale of the Testing MSE is orders of magnitude higher than that of the training.

## Perform Ridge and Lasso regression

Use all the data (normalized features and dummy categorical variables) and perform Lasso and Ridge regression for both! Each time, look at R-squared and MSE.

## Lasso

With default parameter (alpha = 1)

In [22]:
# Your code here
from sklearn.linear_model import Lasso, Ridge
lasso = Lasso()
lasso.fit(X_train, y_train)
print('Training r^2:', lasso.score(X_train, y_train))
print('Testing r^2:', lasso.score(X_test, y_test))
print('Training MSE:', mean_squared_error(y_train, lasso.predict(X_train)))
print('Testing MSE:', mean_squared_error(y_test, lasso.predict(X_test)))

Training r^2: 0.9376760335945125
Testing r^2: 0.6935195984938209
Training MSE: 405739123.02231413
Testing MSE: 1737123021.7523525


With a higher regularization parameter (alpha = 10)

In [23]:
# Your code here
lasso = Lasso(10)
lasso.fit(X_train, y_train)
print('Training r^2:', lasso.score(X_train, y_train))
print('Testing r^2:', lasso.score(X_test, y_test))
print('Training MSE:', mean_squared_error(y_train, lasso.predict(X_train)))
print('Testing MSE:', mean_squared_error(y_test, lasso.predict(X_test)))

Training r^2: 0.9361224686057488
Testing r^2: 0.6844062545819503
Training MSE: 415853082.9073129
Testing MSE: 1788777220.3133593


## Ridge

With default parameter (alpha = 1)

In [24]:
# Your code here
ridge = Ridge()
ridge.fit(X_train, y_train)
print('Training r^2:', ridge.score(X_train, y_train))
print('Testing r^2:', ridge.score(X_test, y_test))
print('Training MSE:', mean_squared_error(y_train, ridge.predict(X_train)))
print('Testing MSE:', mean_squared_error(y_test, ridge.predict(X_test)))

Training r^2: 0.9337100272101158
Testing r^2: 0.6743103916888455
Training MSE: 431558467.4112213
Testing MSE: 1846000311.1533577


With default parameter (alpha = 10)

In [25]:
# Your code here# Your code here
ridge = Ridge(10)
ridge.fit(X_train, y_train)
print('Training r^2:', ridge.score(X_train, y_train))
print('Testing r^2:', ridge.score(X_test, y_test))
print('Training MSE:', mean_squared_error(y_train, ridge.predict(X_train)))
print('Testing MSE:', mean_squared_error(y_test, ridge.predict(X_test)))

Training r^2: 0.9189748568643502
Testing r^2: 0.6853219803653536
Training MSE: 527486814.6986449
Testing MSE: 1783586910.1592846


## Look at the metrics, what are your main conclusions?   

Conclusions here

## Compare number of parameter estimates that are (very close to) 0 for Ridge and Lasso

Compare with the total length of the parameter space and draw conclusions!

In [26]:
# number of Ridge params almost zero
print(sum(abs(ridge.coef_)<10**(-10)))

1


In [27]:
# number of Lasso params almost zero
print(sum(abs(lasso.coef_)<10**(-10)))

44


Lasso was very effective to essentially perform variable selection and remove about 25% of the variables from your model!

In [28]:
# your code here
len(lasso.coef_)

246

In [30]:
44/246

0.17886178861788618

## Summary

Great! You now know how to perform Lasso and Ridge regression.